In [17]:
import time

from gensim.models import Word2Vec
from konlpy.tag._komoran import Komoran
import numpy as np

# Embedding
용어:
  - 임베딩(embedding): 문장이나 단어 등 자연어 요소를 수치 혹은 벡터로 만드는 일.
    - 임베딩 벡터: 그렇게 만들어진 벡터.
    - {문장,단어} 임베딩: {문장,단어}를 임베딩 벡터로 만드는 경우.
  - 말뭉치(corpus): 특정한 목적을 가지고 언어의 표본을 추출한 집합(Wikipedia).

문장 단위 임베딩
  - 장점: 문맥적 의미를 벡터에 표현할 수 있다.
  - 단점: 데이터가 많이 필요.
단어 단위 임베딩
  - 장점: 적은 비용.
  - 단점: 문맥적 의미 소실('의사'란 단어가 직업 혹은 생각을 칭할 수 있으나 어떤 경우든 같은 벡터가 됨).

이 프로젝트는 단어 임베딩으로 진행한다.


## 원핫 인코딩(one-hot encoding)
벡터 원소들 중 한 곳만 1, 나머지는 0인 원핫 벡터 혹은 희소 벡터(sparse vector)를 만드는 임베딩.
`[가난, 부자, 아시아]`가 있을 때 `가난=[1, 0, 0]`, `부자=[0, 1, 0]`, `아시아=[0, 0, 1]`가
된다.

코퍼스에서 단어를 추출해 유일한 단어들만 모인 '단어 집합'을 구축하고, 단어 집합 내 출현 위치에 따라
임베딩 벡터에서 1을 부여한다. 임베딩 벡터의 길이는 `len(word_set)`이다.

단점:
  - 차원이 급격히 증가할 수 있어 불필요한 메모리 차지.
  - 등장 위치에 따라 의미를 부여하니 단어 의미가 죽어버림.

In [4]:
komoran = Komoran()
sentence = "아버지는 항상 정각에 출근하신다."

In [5]:
# 단어 집합 구축
word_set = {}
nouns = komoran.nouns(sentence)
for word in nouns:
    if word not in word_set:
        word_set[word] = len(word_set)

In [7]:
word_set

{'아버지': 0, '정각': 1, '출근': 2}

In [11]:
one_hot_vectors = {}
len_vec = len(word_set)
vectors = np.eye(len_vec)
for hot_idx, word in enumerate(word_set.keys()):
    one_hot_vectors[word] = vectors[hot_idx]

In [13]:
one_hot_vectors  # [n]의 의미는 n번째 등장했던 단어가 존재하는지만 알려준다.

{'아버지': array([1., 0., 0.]),
 '정각': array([0., 1., 0.]),
 '출근': array([0., 0., 1.])}

## 분산 표현(distributed representation)
- 희소 표현: 벡터 내 대부분이 0(정보 없음)이고 특정 위치에만 정보가 존재하여, 정보가 희소하게 존재한다.
- 분산 표현: 정보가 벡터 내 여러 위치에 분산돼 있단 의미. 이렇게 표현된 벡터를 '분산 벡터'라고 함.

장점:
  - NLP에선 단어 간 의미 관계가 임베딩 벡터에 잘 드러나야 하니 희소보단 분산 표현이 더 적합하다.
    - 벡터 간 유사도 계산이 가능해진다.
  - 컴퓨팅 자원에서도 분산 표현이 밀집도가 높으니 유리하다.

이 프로젝트에서는 Word2Vec 모델로 이런 분산 표현 임베딩을 할 것이다. 문맥(context words: 주변 단어)
으로 목표 단어를 추측하는 신경망 모델이다.

Word2Vec의 학습 방식:
  - CBOW(Continuous Bag-Of-Words): 입력이 문맥, 출력은 목표 단어
  - Skip-gram: 입력은 목표 단어, 출력은 문맥
이렇게 학습된 파라미터를 목표 단어의 임베딩 벡터로 사용하는 것이다.

사용 코퍼스: [Naver sentiment movie corpus](https://github.com/e9t/nsmc)

In [19]:
def read_reviews(file):
    with open(file, "r") as f:
        data = [line.split("\t") for line in f.readlines()]

    return data[1:]  # without header row

In [20]:
# 1. 리뷰 파일 읽기
corpus_file = "data/corpus/nsmc/ratings.txt"
reviews = read_reviews(corpus_file)

In [23]:
reviews[0]  # [ID, review, rating]

['8112052', '어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '1\n']

In [24]:
# 2. 명사 토큰 추출(시간 걸리는 작업)
noun_tokens_by_review = []
for line in reviews:
    num_id, document, rating = line
    nouns = komoran.nouns(document)
    noun_tokens_by_review.append(nouns)

In [28]:
line_num = 1

print(f"원문: {reviews[line_num][1]}")
print(f"명사들: {noun_tokens_by_review[line_num]}")

원문: 디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.
명사들: ['디자인', '학생', '외국', '디자이너', '전통', '발전', '문화', '산업', '사실', '우리나라', '시절', '끝', '열정', '노라', '노', '전통', '사람', '꿈', '수', '것', '감사']


In [30]:
# 3. Word2Vec 모델 학습
model = Word2Vec(sentences=noun_tokens_by_review,
                 vector_size=200,  # 벡터 길이
                 window=4,  # 앞뒤로 살필 문맥 크기
                 hs=1,  # 0: `negative`인자가 non-zero면 음수 샘플링, 1: 소프트맥스 사용
                 min_count=2,  # 학습하지 않을 단어의 출현 빈도(3회부터 학습)
                 sg=1)  # 0: CBOW, 1: Skip-gram

In [31]:
# 4. 모델 저장
model_path = "models/nvmc.model"
model.save(model_path)

In [38]:
appeared_words = model.corpus_total_words
distinct_words = model.corpus_count
print(f"{appeared_words:,} 명사 단어 중에서 {distinct_words:,} 개의 말뭉치를 뽑았다.")

1,076,896 명사 단어 중에서 200,000 개의 말뭉치를 뽑았다.


In [42]:
emb_vec = model.wv["사랑"]
print(emb_vec)  # 밀집 표현임을 확인.

[ 0.1896135  -0.25879362  0.03680233  0.09719283 -0.0212954  -0.19035025
  0.14886956  0.3862878   0.07676568  0.26586714 -0.11055886 -0.19067588
  0.09060291  0.27099648 -0.18140654  0.39448726  0.08940478 -0.0972247
 -0.19975305 -0.26682636 -0.06186922  0.19001347 -0.00203858  0.07061061
 -0.265596   -0.12545863  0.0495344  -0.21292473 -0.03824321 -0.08135716
  0.17319039  0.00823351 -0.22402056 -0.51960135  0.01338699  0.4529123
  0.2986559   0.02196982 -0.07260319 -0.37092608 -0.4037158   0.0733239
 -0.19598487  0.01557027  0.37291405  0.25948644 -0.45321357 -0.15969439
  0.509996   -0.23106395 -0.18009372  0.16885254  0.02537337  0.04026189
  0.06998719  0.06692788  0.35140684 -0.1780219   0.0292092   0.0510111
 -0.1032525   0.09403237 -0.15633427  0.16050664  0.09056013 -0.14417246
 -0.33054858  0.10788924 -0.02351671  0.3641776  -0.05931226 -0.20083079
  0.02831833  0.07329393  0.04698088 -0.19710518  0.36676475 -0.10963044
 -0.41987818  0.31984693  0.06986964 -0.22335866 -0.023

In [44]:
# 유사도 값은 [0,1] 범위며 1에 가까울수록 유사함을 의미한다.
sim_love_affection = model.wv.similarity("사랑", "애정")
print(f"사랑 애정: {sim_love_affection}")

sim_cola_love = model.wv.similarity("콜라", "사랑")
print(f"사랑 콜라: {sim_cola_love}")

사랑 애정: 0.5552906394004822
사랑 콜라: 0.24324744939804077


In [49]:
thesaurus_history =  model.wv.most_similar("역사")
print(f"'역사'와 유사한 단어 10개: {thesaurus_history}")

thesaurus_winter = model.wv.most_similar("겨울")
print(f"'겨울'  유사한 단어 10개: {thesaurus_winter}")
# ('여름', 0.5729169845581055)로 결과가 완전하지 않음을 알 수 있다.

'역사'와 유사한 단어 10개: [('왜곡', 0.6893980503082275), ('김치양', 0.6817640066146851), ('BEST', 0.5832754373550415), ('세계사', 0.5807084441184998), ('승자', 0.5728675127029419), ('한국전', 0.5714324116706848), ('역사관', 0.5711150169372559), ('고증', 0.5610010623931885), ('제정', 0.5570429563522339), ('퇴행', 0.5564497113227844)]
'겨울'와 유사한 단어 10개: [('햇살', 0.5769104957580566), ('여름', 0.5729169845581055), ('가을', 0.5688246488571167), ('.tv', 0.5612796545028687), ('강변', 0.5602102875709534), ('방학', 0.5600340962409973), ('멜로디.', 0.5594018697738647), ('그랑블루', 0.5590022802352905), ('하오', 0.5574026107788086), ('힐링', 0.5568239092826843)]
